In [ ]:
!pip install --upgrade torch torchvision

     |████████████████████████████████| 831.4 MB 1.7 kB/s  eta 0:00:01    |██▎                             | 60.4 MB 36.1 MB/s eta 0:00:22     |███                             | 80.7 MB 36.1 MB/s eta 0:00:21     |█████████████████▋              | 457.6 MB 21.9 MB/s eta 0:00:18     |██████████████████              | 470.1 MB 21.9 MB/s eta 0:00:17     |█████████████████████████████   | 752.3 MB 43.8 MB/s eta 0:00:02     |█████████████████████████████▋  | 770.6 MB 43.8 MB/s eta 0:00:02     |██████████████████████████████  | 777.3 MB 12.7 MB/s eta 0:00:05     |██████████████████████████████  | 780.5 MB 12.7 MB/s eta 0:00:05     |██████████████████████████████▍ | 789.3 MB 12.7 MB/s eta 0:00:04     |██████████████████████████████▌ | 792.5 MB 12.7 MB/s eta 0:00:04
     |████████████████████████████████| 22.1 MB 349 kB/s  eta 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 1.7.0
    Uninstalling torch-1.7.0:
      Successfully uninstalled torch-1.7.0
  Attempting u

In [ ]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-geometric
!pip install -q git+https://github.com/snap-stanford/deepsnap.git

In [ ]:
import torch
import warnings
import networkx as nx
import matplotlib.pyplot as plt
import scipy
from scipy import io
from pylab import *
from copy import deepcopy
from deepsnap.graph import Graph
from deepsnap.batch import Batch
from deepsnap.dataset import GraphDataset
from torch.utils.data import DataLoader
from torch_geometric.nn import VGAE
import copy
import torch.nn as nn
import networkx as nx
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_auc_score,precision_score,recall_score
from torch_geometric.datasets import Planetoid, TUDataset
from torch_geometric.nn import SAGEConv,GCNConv,GATConv,SGConv,GENConv,HypergraphConv
from tqdm.notebook import tqdm
import numpy as np
# Disable DeepSNAP warnings for clearer printout in the tutorial
warnings.filterwarnings("ignore")

In [ ]:
### Loading Matrix
x = scipy.sparse.load_npz('deep_patient_matrix.npz')
x = x.astype(np.float64)

In [ ]:
## Building Graph
G = nx.from_scipy_sparse_matrix(x).to_undirected()

In [ ]:
## assigning patient = [0,1]  and disease = [1,0] and then adding it as a input feature
num = 30000  ## no of patient
pat = torch.ones(num,2)
pat[:,0] = torch.zeros(num,)
dis = torch.ones(47364,2)
dis[:,1] = torch.zeros(47364,)
attr = torch.ones(num+47364,2)
attr[:num] = pat
attr[num:] = dis

In [ ]:
## converting to dictionary 
attribute = {}
for i in range(num+47364):
    attribute[i] = attr[i]

In [ ]:
## updating those attributes as node feature
nx.set_node_attributes(G,attribute,"node_feature")

In [ ]:
class LinkPredModel(torch.nn.Module):
    def __init__(self, input_size, hidden_dim,num_layers=5,output_dim=128):
        super(LinkPredModel, self).__init__()

        self.loss_fn = torch.nn.BCEWithLogitsLoss()
        self.list = []
        self.list.append(input_dim)
        for i in range(num_layers-2):
           self.list.append(hidden_dim)
        self.list.append(output_dim)
        self.convs = torch.nn.ModuleList([SAGEConv(self.list[i],self.list[i+1])for i in range(num_layers-1)]) ## use GCNConv,SGConv,SAGEConv for respective models
        self.bns = torch.nn.ModuleList([torch.nn.BatchNorm1d(self.list[i+1])for i in range(num_layers-1)])
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, batch,test = False):
        x, edge_index, edge_label_index = batch.node_feature, batch.edge_index, batch.edge_label_index
        x = x.float()
        for i in range(len(self.convs)-1):
            x = self.convs[i](x,edge_index)
            x = self.bns[i](x)
            x = torch.nn.functional.relu(x)
            x = self.dropout(x)   
        x = self.convs[i+1](x,edge_index)
        if test == True :  ## if in test phase then return all the embeddings of nodes otherwise find scores of given edges
            return x
        nodes_first = torch.index_select(x, 0, edge_label_index[0,:].long())
        nodes_second = torch.index_select(x, 0, edge_label_index[1,:].long())
        pred = torch.sum(nodes_first * nodes_second, dim=-1)
        return pred
    


In [ ]:
## BCE Loss Function
bce_loss = torch.nn.BCEWithLogitsLoss()
def bceloss( pred, label):
        loss = bce_loss(pred,label)
        return loss
def f1_loss(pred,label):
        y = torch.sigmoid(pred)
        tp = torch.sum(label*y)
        fp = torch.sum((1-label)*y)
        fn = torch.sum((1-y)*label)
        soft_f1 = (2*tp+1e-16) / (tp + 0.3*fn+ 1.7*fp + 1e-16)
        sz = y.shape[0]
        return 1-soft_f1
def delta_loss(pred,label,delta=0.4):  ## Max Margin Loss
    pred = torch.sigmoid(pred)
    pos = (label)*pred
    neg = (1-label)*pred
    score = delta+neg-pos
    score = torch.maximum(score,torch.zeros(1).to('cuda'))
    score = torch.mean(score)
    return score

In [ ]:
def train(model, dataloaders, optimizer, args):
    val_max = 0
    best_model = model

    for epoch in tqdm(range(1, args["epochs"])):
        for i, batch in enumerate(dataloaders['train']):
            batch.to(args["device"])
            model.train()
            optimizer.zero_grad()
            pred = model(batch)
            loss = bceloss(pred,batch.edge_label.type(pred.dtype))
            loss.backward()
            optimizer.step()

            log = 'Epoch: {:03d}, Train_prec: {:.4f},Train_rec: {:.4f}, Val_prec: {:.4f},Val_rec: {:.4f}, Test_prec: {:.4f},Test_rec: {:.4f}, Loss: {:.5f}'
            precision_train,recall_train = test(model, dataloaders['train'], args)
            precision_val,recall_val = test(model, dataloaders['val'], args)
            precision_test,recall_test = test(model, dataloaders['test'], args)

            print(log.format(epoch, precision_train,recall_train, precision_val,recall_val, precision_test,recall_test, loss.item()))
            if val_max < precision_val:
                val_max = precision_val
                best_model = copy.deepcopy(model)
    return model

def test(model, dataloader,args,th =0.55):
    model.eval()
    score = 0
    score1 = 0
    score2 = 0
    num_batches = 0
    for batch in dataloader:
        batch.to(args["device"])
        pred = model(batch)
        pred2 = torch.sigmoid(pred)
        pred = (pred2>th)*1
        score1 += precision_score(batch.edge_label.flatten().cpu().numpy(), pred.flatten().data.cpu().numpy())
        score2 += recall_score(batch.edge_label.flatten().cpu().numpy(), pred.flatten().data.cpu().numpy())
        score += roc_auc_score(batch.edge_label.flatten().cpu().numpy(), pred2.flatten().data.cpu().numpy())
        num_batches += 1
    score1 /= num_batches
    score2 /= num_batches
    score /= num_batches
    print("Auc = ", score)
    return score1,score2



In [ ]:
## Loading Dataset and splitting into train,val and test data
dataset = GraphDataset(
        Graph(G),
        task='link_pred',
        edge_train_mode="disjoint"
    )
datasets = {}
datasets['train'], datasets['val'], datasets['test']= dataset.split(
            transductive=True, split_ratio=[0.9, 0.04, 0.06])
dataloaders = {split: DataLoader(
            ds, collate_fn=Batch.collate([]),
            batch_size=1, shuffle=(split=='train'))
            for split, ds in datasets.items()}

In [ ]:
### GCN Conv - Hidden dim = 128
### SG Conv - Hidden dim = 128
### Graph sage - Hidden dim = 64
args = {
    "device" : 'cuda' if torch.cuda.is_available() else 'cpu',
    "hidden_dim" : 64,  
    'num_layers': 4,
    "epochs" : 10,
}

In [ ]:
input_dim = datasets['train'].num_node_features
num_classes = datasets['train'].num_edge_labels
model = LinkPredModel(input_dim, args["hidden_dim"]).to(args["device"])

In [ ]:
## Loading Pre trained Model
### Gcn - gcn.pth
### SGConv - sgconv.pth
## graphsage - graphsage.pth
model = torch.load('Model/gcn.pth')

### Training

In [ ]:
input_dim = datasets['train'].num_node_features
num_classes = datasets['train'].num_edge_labels
model = LinkPredModel(input_dim, args["hidden_dim"]).to(args["device"])
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
best_model = train(model, dataloaders, optimizer, args)
log = "Train: {:.4f}, Val: {:.4f}, Test: {:.4f}"
best_train_prec,best_train_rec = test(best_model, dataloaders['train'], args)
best_val_prec,best_val_rec = test(best_model, dataloaders['val'],args)
best_test_prec,best_test_rec = test(best_model, dataloaders['test'], args)
print(log.format(best_train_prec,best_train_rec,best_val_prec,best_val_rec,best_test_prec,best_test_rec))

  0%|          | 0/9 [00:00<?, ?it/s]

Auc =  0.7000026728941713
Auc =  0.6971178849486117
Auc =  0.6944624922886302
Epoch: 001, Train_prec: 0.6169,Train_rec: 0.9960, Val_prec: 0.6077,Val_rec: 0.9968, Test_prec: 0.6045,Test_rec: 0.9967, Loss: 0.60418
Auc =  0.7245704691171833
Auc =  0.7217507358231331
Auc =  0.7219924923093036
Epoch: 002, Train_prec: 0.6168,Train_rec: 0.9960, Val_prec: 0.6077,Val_rec: 0.9968, Test_prec: 0.6045,Test_rec: 0.9967, Loss: 0.60469
Auc =  0.7499456175517525
Auc =  0.742483280257166
Auc =  0.741625625794974
Epoch: 003, Train_prec: 0.6163,Train_rec: 0.9960, Val_prec: 0.6077,Val_rec: 0.9968, Test_prec: 0.6045,Test_rec: 0.9967, Loss: 0.60406
Auc =  0.7727191172397878
Auc =  0.7712639750259999
Auc =  0.7657975774099868
Epoch: 004, Train_prec: 0.6167,Train_rec: 0.9960, Val_prec: 0.6077,Val_rec: 0.9968, Test_prec: 0.6045,Test_rec: 0.9967, Loss: 0.60394
Auc =  0.7352137841959624
Auc =  0.7263089244281665
Auc =  0.7222179355900185
Epoch: 005, Train_prec: 0.6168,Train_rec: 0.9960, Val_prec: 0.6077,Val_rec: 

### Testing

In [ ]:
with torch.no_grad():
    model.eval()    
    for batch in dataloaders['test']:
        batch.to(args["device"])
        pred2 = model(batch,test = True)

In [ ]:
mat = x ## adjacency matrix
nrecommend = 100
total_recommendable = 3021969
hit = 0 ## Counting no of hit

for k in tqdm(range(30)): ## 1000 patient at a time so total 30 times
    pred = pred2[(k)*1000:(k+1)*1000].matmul(pred2[30000:].t()) ### dot product of patient score and disease score (pred2[30000:]) 
    sorted, indices = torch.sort(pred)  ## sorting scores to get top k scores
    for i in range(1000):
        
       for j in range(nrecommend):   ### top 100 scores
          if mat[i,indices[i][47363-j].item()+30000] == 1 : ## Checking if it's a hit
            hit=hit+1
precision = hit/(30000*nrecommend)
recall = hit/total_recommendable
print("no of hits = ",hit,"Recall = ", recall,"Precison = ",precision)

  0%|          | 0/30 [00:00<?, ?it/s]

no of hits =  775978 Recall =  0.2567789411473116 Precison =  0.25865933333333335


In [ ]:
### prediction for patient 
id = 1 ## id or index no of patient
top_k = 10 ## top k predictions
icd_code = np.load('Models/icd_codes.npy',allow_pickle = True)
pred = pred2[id].matmul(pred2[30000:].t())
sorted, indices = torch.sort(pred)
for j in range(top_k-1,-1,-1):
   idx = indices[47363-j].item()
   if mat[idx+30000] == 1 :
       print(icd_code[idx],"Hit")
   else:
       print(icd_code[idx],"Miss")



## Variational Graph Autoencoder

In [ ]:
from torch_geometric.nn import VGAE

In [ ]:
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, input_dim, output_dim,hidden_dim,num_layers):
        super(VariationalGCNEncoder, self).__init__()
        self.list = []
        self.list.append(input_dim)
        for i in range(num_layers-2):
           self.list.append(hidden_dim)
        self.list.append(output_dim)
        self.convs = torch.nn.ModuleList([GCNConv(self.list[i],self.list[i+1])for i in range(num_layers-1)])
        self.conv_mu = GCNConv( out_channels, out_channels//2)
        self.conv_logstd = GCNConv( out_channels, out_channels//2)
        self.bns = torch.nn.ModuleList([torch.nn.BatchNorm1d(self.list[i+1])for i in range(num_layers-1)])
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, x, edge_index):
        for i in range(len(self.convs)-1):
            x = self.convs[i](x,edge_index)
            x = self.bns[i](x)
            x = torch.nn.functional.relu(x)
            x = self.dropout(x)
        x = self.convs[i+1](x,edge_index)
        x = torch.nn.functional.relu(x)
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [ ]:
def train(model, dataloaders, optimizer, args):
    val_max = 0
    best_model = model

    for epoch in tqdm(range(1, args["epochs"])):
        for i, batch in enumerate(dataloaders['train']):
            batch.to(args["device"])
            model.train()
            optimizer.zero_grad()
            
            x, edge_index, edge_label_index = batch.node_feature, batch.edge_index, batch.edge_label_index

            x = x.float()
            z = model.encode(x, edge_index)
            
            half = int(batch.edge_label.shape[0]/2)
            pos_index = edge_label_index[:,0:half]
            neg_index = edge_label_index[:,half:]
            
            loss = model.recon_loss(z, pos_index,neg_index)
            loss = loss + (1 / batch.node_feature.shape[0]) * model.kl_loss()  
            loss.backward()
            optimizer.step()
            

            log = 'Epoch: {:03d}, Train_prec: {:.4f},Train_rec: {:.4f}, Val_prec: {:.4f},Val_rec: {:.4f}, Test_prec: {:.4f},Test_rec: {:.4f}, Loss: {:.5f}'
            precision_train,recall_train = test(model, dataloaders['train'], args)
            precision_val,recall_val = test(model, dataloaders['val'], args)
            precision_test,recall_test = test(model, dataloaders['test'], args)

            print(log.format(epoch, precision_train,recall_train, precision_val,recall_val, precision_test,recall_test, loss.item()))
            if val_max < precision_val:
                val_max = precision_val
                torch.save(model,'vgae.pth') 
    return model
def test(model, dataloader,args,th =0.55):
    model.eval()
    score = 0
    score1 = 0
    score2 = 0
    num_batches = 0
    for batch in dataloader:
        batch.to(args["device"])
        x, edge_index, edge_label_index = batch.node_feature, batch.edge_index, batch.edge_label_index
        x = x.float()
        z = model.encode(x, edge_index)
        pred = model.decoder(z,batch.edge_label_index,sigmoid=False)
        pred2 = torch.sigmoid(pred)
        pred = (pred2>th)*1
        score1 += precision_score(batch.edge_label.flatten().cpu().numpy(), pred.flatten().data.cpu().numpy())
        score2 += recall_score(batch.edge_label.flatten().cpu().numpy(), pred.flatten().data.cpu().numpy())
        score += roc_auc_score(batch.edge_label.flatten().cpu().numpy(), pred2.flatten().data.cpu().numpy())
        num_batches += 1
    score1 /= num_batches
    score2 /= num_batches
    score /= num_batches
    print("Auc = ", score)
    return score1,score2


In [ ]:
args = {
    "device" : 'cuda' if torch.cuda.is_available() else 'cpu',
    "hidden_dim" : 64,
    'num_layers': 4,
    "epochs" : 70,
}

In [ ]:
out_channels = 64
num_features = datasets['train'].num_node_features
epochs = 70


model = VGAE(VariationalGCNEncoder(num_features, out_channels,args["hidden_dim"],args["num_layers"]))  # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.07, momentum=0.9, weight_decay=5e-4)


In [ ]:
## Loading Model
model = torch.load('Model/vgae.pth')

### Training

In [ ]:
best_model = train(model, dataloaders, optimizer, args)
log = "Train: {:.4f}, Val: {:.4f}, Test: {:.4f}"
best_train_prec,best_train_rec = test(best_model, dataloaders['train'], args)
best_val_prec,best_val_rec = test(best_model, dataloaders['val'],args)
best_test_prec,best_test_rec = test(best_model, dataloaders['test'], args)
print(log.format(best_train_prec,best_train_rec,best_val_prec,best_val_rec,best_test_prec,best_test_rec))

## Testing

In [ ]:
with torch.no_grad():
    model.eval()    
    for batch in dataloaders['test']:
        batch.to(args["device"])
        x, edge_index, edge_label_index = batch.node_feature,batch.edge_index, batch.edge_label_index
        x = x.float()
        z = model.encode(x, edge_index)
        pred2 = model.decoder(z,batch.edge_label_index,sigmoid=True)
        z = torch.sigmoid(z)

In [ ]:
## Loading matrix
mat = scipy.sparse.load_npz('deep_patient_matrix.npz')

In [ ]:
nrecommend = 100
total_recommendable = 3021969
hit = 0 ## Counting no of hit
pred2 = z

for k in tqdm(range(30)): ## 1000 patient at a time so total 30 times
    pred = pred2[(k)*1000:(k+1)*1000].matmul(pred2[30000:].t()) ### dot product of patient score and disease score (pred2[30000:]) 
    sorted, indices = torch.sort(pred)  ## sorting scores to get top k scores
    for i in range(1000):
        
       for j in range(nrecommend):   ### top 100 scores
          if mat[i,indices[i][47363-j].item()+30000] == 1 : ## Checking if it's a hit
            hit=hit+1
precision = hit/(30000*nrecommend)
recall = hit/total_recommendable
print("no of hits = ",hit,"Recall = ", recall,"Precison = ",precision)

In [ ]:
### prediction for patient 
id = 1 ## id or index no of patient
top_k = 10 ## top k predictions
icd_code = np.load('Models/icd_codes.npy',allow_pickle = True)
pred = z[id].matmul(z[30000:].t())
sorted, indices = torch.sort(pred)
for j in range(top_k-1,-1,-1):
   idx = indices[47363-j].item()
   if mat[idx+30000] == 1 :
       print(icd_code[idx],"Hit")
   else:
       print(icd_code[idx],"Miss")